# Orchestration with GenAI Hub

This notebook demonstrates setting up data masking and content filtering, configuring an orchestration pipeline, and querying multiple LLM models with GenAI Hub.

Installing the required packages

The code imports required libraries, reads credentials from a creds.json file, and sets environment variables for authentication and API access

In [2]:
import time
import json
import os
from IPython.display import clear_output
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from enum import Enum
 
# Inline credentials
with open('creds.json') as f:
    credCF = json.load(f)

# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "grounding" 
    }

    for key, value in env_vars.items():
        os.environ[key] = value
        print(value)

# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )

ai_core_client = create_ai_core_client(credCF)

https://anuragvxxx.authentication.eu10.hana.ondemand.com/oauth/token
sb-xxxxxxx
10xxxxx=
https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2
grounding


In [3]:
YOUR_API_URL = "https://api.ai.prodeuonly.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/db4b41879faa1ed4"

## Basic Orchestration Pipeline

Now that you have YOUR_DEPOLYMENT_URL, let's walk through a basic orchestration pipeline.

In [7]:

from gen_ai_hub.orchestration.utils import load_text_file 
# Load the support request file content 
support_request_path = r"C:\Users\C5384965\OneDrive - SAP SE\2026\jan-26\06-01-26\ai-core-orchestration-consumption-opt\support-request.txt" # Specify the correct path to the file 
support_request = load_text_file(support_request_path) 
# Print the content to verify it has been loaded 
print(support_request)

"Subject: Bestellung #1234567890 VerspÃ¤tet - John Johnson Nachricht: Halle, ich schreibe ihnen um mich nach dem Status meiner Bestellung mit der Bestellnr. +1234567890 zu erkundigen. Die Lieferung war eigentlich fÃ¼r gestern geplant, ist bisher jedoch nicht erfolgt. Mein Name ist John Johnson und meine Lieferadresse lautet 125 Cole Meadows Drive Palo Alto, California 94301. Bitte lassen Sie mich per Telefon unter der Nummer +1 505802 2172 wissen, wann ich mit meiner Lieferung rechnen kann. Danke!"


# Step 1: Templating

Explanation of Templating Code

This code defines a template for an AI assistant using orchestration configuration. The `Template` object is set up with system and user messages to guide the assistant’s response behavior. 

Key Components:
- **SystemMessage**: Sets a predefined instruction for the AI assistant. This message typically includes the assistant's role and any specific guidelines it should follow.
- **UserMessage**: Represents the user's input and how it is structured in the conversation.
  
In this revised prompt, only queries are passed to the assistant without any additional context. The AI is expected to respond based solely on the provided input.


In [8]:
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

# Define the sentiment analysis template
template = Template(
    messages=[
        SystemMessage(
            """You are a customer support assistant. Analyze the sentiment of the user request provided and return whether the sentiment is positive, neutral, or negative. Also provide a one-line justification for your classification."""
        ),
        UserMessage(
            "Please analyze the sentiment of the following support request: {{?support_text}}"
        ),
    ],
    defaults=[
        TemplateValue(name="support_text", value="User is unhappy with the latest update and facing usability issues."),
    ],
)


# Step 2: Define the LLM -list of models

The LLM class is used to configure and initialize a model for generating text based on specific parameters. In this example, we'll use the list of models to perform the content creation task.

ℹ️Note that virtual deployment of the model is managed automatically by the Orchestration Service, so no additional deployment setup is required on your part.

In [17]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(name="gpt-5-nano", parameters={"max_completion_tokens": 512})

This configuration initializes the model to use the list of llm models with the latest updates. The model will generate responses up to 256 tokens in length and produce more predictable and focused output due to the low temperature setting.

### Data Masking

The Data Masking Module anonymizes or pseudonymizes personally identifiable information (PII) before it is processed by the LLM module. When data is anonymized, all identifying information is replaced with placeholders (e.g., MASKED_ENTITY), and the original data cannot be recovered, ensuring that no trace of the original information is retained. In contrast, pseudonymized data is substituted with unique placeholders (e.g., MASKED_ENTITY_ID), allowing the original information to be restored if needed. In both cases, the masking module identifies sensitive data and replaces it with appropriate placeholders before further processing.

In [13]:
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, ProfileEntity

# Apply data masking to sensitive information in the resume
data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # or MaskingMethod.PSEUDONYMIZATION
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON,
                ProfileEntity.ORG,
                ProfileEntity.LOCATION
            ]
        )
    ]
)


### Content Filtering

The Content Filtering Module can be configured to filter both the input to the LLM module (input filter) and the output generated by the LLM (output filter). The module uses predefined classification services to detect inappropriate or unwanted content, allowing flexible configuration through customizable thresholds. These thresholds can be set to control the sensitivity of filtering, ensuring that content meets desired standards before it is processed or returned as output.

In [11]:

from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter, AzureThreshold
from gen_ai_hub.orchestration.models.llama_guard_3_filter import LlamaGuard38bFilter

input_filter= AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                  violence=AzureThreshold.ALLOW_SAFE,
                                  self_harm=AzureThreshold.ALLOW_SAFE,
                                  sexual=AzureThreshold.ALLOW_SAFE)
input_filter_llama = LlamaGuard38bFilter(hate=True)
output_filter = AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                   violence=AzureThreshold.ALLOW_SAFE_LOW,
                                   self_harm=AzureThreshold.ALLOW_SAFE_LOW_MEDIUM,
                                   sexual=AzureThreshold.ALLOW_ALL)
output_filter_llama = LlamaGuard38bFilter(hate=True)


### Translation

Translation module can be used to translate text from one language to another. You can use this module to translate input text before it is processed by the LLM module, or to translate the output generated by the LLM module. The translation module uses the SAP Document Translation service to perform the translation.

In [16]:
from gen_ai_hub.orchestration.models.translation.translation import InputTranslationConfig, OutputTranslationConfig
from gen_ai_hub.orchestration.models.translation.sap_document_translation import SAPDocumentTranslation

input_config = InputTranslationConfig(source_language="de-DE", target_language="en-US")
output_config = OutputTranslationConfig(source_language="en-US", target_language="de-DE")

translation_module = SAPDocumentTranslation(
    input_translation_config=input_config,
    output_translation_config=output_config)

# Step 3: Create the Orchestration Configuration

The OrchestrationConfig class is used to create a configuration that integrates various components, such as templates and llm models, into a unified orchestration setup. This configuration specifies how these components work together to achieve the desired workflow.

In [ ]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.content_filtering import InputFiltering, OutputFiltering, ContentFiltering
# Define content filtering
content_filtering = ContentFiltering(
    input_filtering=InputFiltering(filters=[input_filter, input_filter_llama]),
    output_filtering=OutputFiltering(filters=[output_filter, output_filter_llama]),
)

config = OrchestrationConfig(
    template=template,
    llm=llm,  
    filtering=content_filtering, 
    data_masking = data_masking,
    translation=translation_module 
)

# Step 4: Run the Orchestration Request

The OrchestrationService class is used to interact with the orchestration service by providing a configuration and invoking its operations. This service handles the execution of workflows defined by the provided configuration and processes inputs accordingly.

In [22]:
from gen_ai_hub.orchestration.service import OrchestrationService

orchestration_service = OrchestrationService()

# Run orchestration with the provided input (for example, candidate resume content)
result = orchestration_service.run(config=config, template_values=[
        TemplateValue(name="support_text", value=support_request)  
    ])  


In [25]:
# Extract the response content
response = result.orchestration_result.choices[0].message.content
print(response)

Negativ – Die Nachricht drückt Frustration und Dringlichkeit aufgrund einer verspäteten Lieferung aus und fordert eine zeitnahe Nachverfolgung an.
